In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import os
import numpy as np
import pandas as pd

In [4]:
import lib
import plotting

In [5]:
from configparser import ConfigParser
cfg = ConfigParser()
cfg.read("config.cfg")

['config.cfg']

In [6]:
from tqdm._tqdm_notebook import tqdm_notebook as tqdm # we manually import the notebook submodule as the normal one struggles with jupyterlab
tqdm.pandas() # this enables us to use progress_apply instead of apply

In [7]:
results_prefix = "[WIKIDATA]"

sequence_file = os.path.join(cfg.get("directory", "exchange"), results_prefix + "001_clustering.p")
labels_file = os.path.join(cfg.get("directory", "exchange"), "[wikidata]004_labels.p")

In [8]:
df = pd.read_pickle(sequence_file)
df.head()

,length,sequence,user_id,stat_dist,kmeans_2,kmeans_3,kmeans_4,kmeans_5,kmeans_6,kmeans_7,kmeans_8,kmeans_9,kmeans_10
0,1511,"[REVERT_MANUAL, BREAK, CLAIM_CREATE, BREAK, CL...",1,"[0.022260285362644863, 0.01999793118304165, 0....",1,0,3,2,2,4,1,2,2
1,9,"[ENTITY_CREATE, BREAK, SITELINK_ADD, BREAK, ME...",1000036,"[0.020070153072094386, 0.02007015307209438, 0....",0,0,2,4,0,0,0,0,0
2,7,"[SITELINK_ADD, BREAK, SITELINK_ADD, BREAK, SIT...",1000078,"[0.01947397679104996, 0.019473976791049966, 0....",0,2,1,1,1,1,5,3,3
3,9,"[ENTITY_CREATE, BREAK, SITELINK_REMOVE, SITELI...",100008,"[0.02042483183496783, 0.020424831834967825, 0....",0,2,1,1,1,1,5,0,0
4,44,"[SITELINK_ADD, BREAK, SITELINK_ADD, BREAK, SIT...",100012,"[0.01973684210526316, 0.019736842105263157, 0....",0,2,1,1,1,1,5,3,3


In [9]:
labels = pd.read_pickle(labels_file)['label'].values

In [10]:
num_clusters = 8
df_selector = "kmeans_{n}".format(n=num_clusters)

In [11]:
cluster_group = df.groupby(df_selector)

In [19]:
results = []
for i, g in cluster_group:
    actions = g["sequence"].apply(len)
    res = {
        "cluster": i,
        "population": len(g),
        "actions_mean": actions.mean(),
        "actions_median": actions.median(),
        "actions_max": actions.max(),
        "actions_min": actions.min()
    }
    
    for i, r in g.iterrows():
        if len(r['sequence']) == actions.max():
            print(r["user_id"])
    results.append(res)
    
df_res = pd.DataFrame(results)
df_res

145693
133116
300821
2221426
2883061
2211707
28325
372


,actions_max,actions_mean,actions_median,actions_min,cluster,population
0,1873540,749.741952,5.0,2,0,28173
1,9818156,7360.444942,32.0,2,1,11379
2,196642,154.065982,6.0,2,2,2531
3,47055,105.777276,12.0,2,3,10861
4,11681018,6619.027118,11.0,2,4,5052
5,89097,193.374556,13.0,2,5,25051
6,29800,37.302671,3.0,2,6,2696
7,200931,234.700058,5.0,2,7,1737
